# 03 - Memory Chat

学习如何在 LangChain 中实现带记忆的对话系统，让 AI 能够记住之前的对话内容。

## 学习目标
- 理解对话记忆的重要性
- 掌握手动管理对话历史的方法
- 学习不同类型的记忆管理策略
- 实现连续的多轮对话

In [ ]:
# 导入必要的库
import os
from dotenv import load_dotenv
from pydantic import SecretStr

# 加载环境变量
load_dotenv(override=True)

# 检查 API 密钥
if not os.getenv('OPENAI_API_KEY'):
    raise ValueError("请设置 OPENAI_API_KEY 环境变量")

# 导入 LangChain 组件
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

print("✓ 环境和组件导入完成")

In [ ]:
# 从环境变量读取配置
api_key = os.getenv("OPENAI_API_KEY")
base_url = os.getenv("OPENAI_BASE_URL")
model_name = os.getenv("MODEL_NAME", "gpt-3.5-turbo")

# 初始化模型
llm = ChatOpenAI(
    model=model_name,
    temperature=0.7,
    api_key=SecretStr(api_key),
    base_url=base_url,
)

print("✓ 模型初始化完成")

## 1. 手动管理对话历史

In [ ]:
# 创建对话历史列表
chat_history = []
max_history = 10  # 保留最近 10 轮对话

print("✓ 对话历史创建完成")
print(f"最大历史长度：{max_history}")

In [ ]:
# 创建带记忆的提示词模板
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个友好的 AI 助手，能够记住之前的对话内容。"),
    MessagesPlaceholder(variable_name="chat_history"),  # 占位符，用于插入对话历史
    ("human", "{input}")
])

print("✓ 带记忆的提示词模板创建完成")
print(prompt)

In [ ]:
# 创建对话函数
def format_chat_history(history):
    """格式化对话历史"""
    formatted = []
    for msg in history:
        if msg["type"] == "human":
            formatted.append(HumanMessage(content=msg["content"]))
        elif msg["type"] == "ai":
            formatted.append(AIMessage(content=msg["content"]))
    return formatted

def chat(conversation, user_input, history):
    """处理对话并更新历史"""
    response = conversation.invoke({
        "input": user_input,
        "chat_history": format_chat_history(history)
    })
    
    # 更新历史
    history.append({"type": "human", "content": user_input})
    history.append({"type": "ai", "content": response})
    
    # 限制历史长度
    while len(history) > max_history * 2:
        history.pop(0)
        history.pop(0)
    
    return response

# 创建对话链
conversation = (
    RunnablePassthrough.assign(
        chat_history=lambda x: format_chat_history(x["chat_history"])
    )
    | prompt
    | llm
    | StrOutputParser()
)

print("✓ 对话链创建完成")

## 2. 测试记忆功能

In [ ]:
# 开始对话
print("=== 开始对话 ===")

# 第一轮对话
response1 = chat(conversation, "你好！我叫小明，今年 25 岁。", chat_history)
print(f"用户：你好！我叫小明，今年 25 岁。")
print(f"AI：{response1}")

# 第二轮对话
response2 = chat(conversation, "你还记得我的名字吗？", chat_history)
print(f"\n用户：你还记得我的名字吗？")
print(f"AI：{response2}")

In [ ]:
# 继续对话，测试记忆功能
response3 = chat(conversation, "我还记得我的名字和年龄吗？", chat_history)
print(f"\n用户：我还记得我的名字和年龄吗？")
print(f"AI：{response3}")

In [ ]:
# 再聊几轮，测试窗口限制
additional_questions = [
    "我最喜欢的编程语言是什么？",
    "我的职业是什么？",
    "我有什么爱好？",
    "我住在哪个城市？",
    "你还记得我最初介绍的信息吗？"
]

for question in additional_questions:
    response = chat(conversation, question, chat_history)
    print(f"\n用户：{question}")
    print(f"AI：{response}")

## 3. 查看记忆内容

In [ ]:
# 查看当前记忆中的对话
print("=== 当前记忆中的对话 ===")

for i, msg in enumerate(chat_history):
    msg_type = "用户" if msg["type"] == "human" else "AI"
    content = msg["content"][:50] + "..." if len(msg["content"]) > 50 else msg["content"]
    print(f"{i+1}. {msg_type}：{content}")

print(f"\n总共记忆了 {len(chat_history)} 条消息")

## 4. 完整对话记忆示例

In [ ]:
# 创建另一个对话示例，使用更大的历史限制
full_chat_history = []
full_max_history = 20

def chat_full(conversation, user_input, history, max_len):
    """处理对话并更新历史（完整记忆）"""
    response = conversation.invoke({
        "input": user_input,
        "chat_history": format_chat_history(history)
    })
    
    # 更新历史
    history.append({"type": "human", "content": user_input})
    history.append({"type": "ai", "content": response})
    
    return response

print("✓ 完整记忆对话系统创建完成")

In [ ]:
# 测试完整记忆
print("=== 完整记忆对话测试 ===")

questions = [
    "我想学习 Python，有什么建议吗？",
    "第一个项目应该做什么？",
    "需要掌握哪些基础知识？",
    "你刚才推荐的第一个项目是什么？"
]

for question in questions:
    response = chat_full(conversation, question, full_chat_history, full_max_history)
    print(f"\n用户：{question}")
    print(f"AI：{response}")

## 5. 记忆管理功能

In [ ]:
# 演示记忆的保存和加载
print("=== 记忆管理演示 ===")

# 查看记忆内容
print("\n当前完整记忆：")
for i, msg in enumerate(full_chat_history):
    msg_type = "用户" if msg["type"] == "human" else "AI"
    content = msg["content"][:50] + "..." if len(msg["content"]) > 50 else msg["content"]
    print(f"{i+1}. {msg_type}：{content}")

# 清空记忆
print(f"\n清空前有 {len(full_chat_history)} 条消息")
full_chat_history.clear()
print(f"清空后有 {len(full_chat_history)} 条消息")

## 6. 实际应用场景

In [ ]:
# 创建一个专业的客服对话示例
service_chat_history = []
service_max_history = 10

service_prompt = ChatPromptTemplate.from_messages([
    ("system", """
    你是一个专业的客服代表。
    你的职责：
    - 友好耐心地回答客户问题
    - 记住客户的历史问题和偏好
    - 提供准确的产品信息
    - 保持专业语调
    """),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

service_conversation = (
    RunnablePassthrough.assign(
        chat_history=lambda x: format_chat_history(x["chat_history"])
    )
    | service_prompt
    | llm
    | StrOutputParser()
)

print("✓ 客服对话系统创建完成")

In [ ]:
# 模拟客服对话
customer_queries = [
    "你好，我想了解一下你们的产品",
    "有什么功能？",
    "价格是多少？",
    "支持退款吗？",
    "刚才说的价格具体是多少？"
]

print("=== 客服对话模拟 ===")
for query in customer_queries:
    response = chat_full(service_conversation, query, service_chat_history, service_max_history)
    print(f"\n客户：{query}")
    print(f"客服：{response}")

## 总结

在这个示例中，我们学习了：

1. **手动管理历史**: 使用列表存储对话历史
2. **格式化函数**: 将历史转换为消息对象
3. **MessagesPlaceholder**: 在提示词中为对话历史预留位置
4. **记忆管理**: 保存、查看、清空记忆内容
5. **实际应用**: 客服对话系统等场景

### 关键概念
- **历史管理**: 使用列表存储对话历史
- **窗口限制**: 控制历史长度以节省 token
- **消息格式**: 使用 HumanMessage 和 AIMessage
- **动态更新**: 每次对话后更新历史

### 选择建议
- **短期对话**: 使用较小的历史限制（10-20 条）
- **长期对话**: 使用较大的历史限制（50-100 条）
- **客服场景**: 较大的历史值（20-40 条）
- **聊天机器人**: 较小的历史值（10-20 条）

### 下一步
尝试实现更复杂的记忆策略，比如摘要记忆、知识图谱记忆等！